# Test notebook for loading in in NetCDF of sentinel 1

In [ ]:
import xarray as xr
from pathlib import Path
import hvplot.xarray
import rasterio
import rioxarray
import numpy as np

In [ ]:
sentinel_folder = Path(r'..\data\g0_020m')
testdata = xr.open_dataset(sentinel_folder/'20141118_S1A_D_110.nc')
testdata

In [ ]:
print(testdata.rio.crs)

In [ ]:
#testdata = testdata.transpose('lat','lon')

In [ ]:
testdata.rio.write_crs("EPSG:4326", inplace = True)

In [ ]:
testdata

In [ ]:
#testdata.rio.reproject(dst_crs="EPSG:31370")
testdata.rio.crs

In [ ]:
#testdata['g0vv'].plot(x = 'lon', y = 'lat')

In [ ]:
from tkinter import PROJECTING
import cartopy.crs as ccrs
map_projection = ccrs.Globe()
#testdata['g0vv'].plot(transform = map_projection)

In [ ]:
#coarsening van de projecties


In [ ]:
#testdata['g0vv'].hvplot(cmap = 'cividis', crs = ccrs.epsg(4236), projection = ccrs.epsg(4326))
#testdata['g0vv'].hvplot(x = 'lat', y = 'lon', cmap = 'cividis', geo = True)

duurt te lang via hv plot! 

## Resampling 

https://corteva.github.io/rioxarray/stable/examples/resampling.html

## doel van de zwalm er uit te clippen 

In [ ]:
import fiona
import rasterio.mask
zwalm_shape = fiona.open("../data/Zwalm_shape/OS266.shp")

In [ ]:
zwalm_shape.crs

nieuw idee: via geopandas omzetten van 31370 naar WGS84 en dan 
`` from rasterio.mask`` en  ``import mask out, _ = mask(data, gdf.geometry, invert=False)``

In [ ]:
import geopandas as gpd
zwalm_gpd_emma = gpd.read_file("../data/Zwalm_shape/OS266.shp") #deze bevat de verschillende substroomgebieden
zwalm_gpd = gpd.read_file("../data/Zwalm_shape/zwalm_shapefile.shp")
zwalm_gpd.plot()
zwalm_gpd_emma.plot()
zwalm_gpd_emma

In [ ]:
omtrekking_zwalm = zwalm_gpd_emma.unary_union
omtrekking_zwalm.area
d = {'PolygonId': 15, 'Area': omtrekking_zwalm.area, 'Subbasin':0,'geometry':omtrekking_zwalm}
zwalm_gpd_emma = zwalm_gpd_emma.append(d, ignore_index= True)
zwalm_gpd_emma

In [ ]:
zwalm_gpd.crs

In [ ]:
zwalm_gpd_emma.crs = 31370
zwalm_gpd_emma.crs

In [ ]:
zwalm_gpd['geometry'] = zwalm_gpd['geometry'].to_crs(epsg = 4326)
zwalm_gpd.plot()
zwalm_gpd_emma['geometry'] = zwalm_gpd_emma['geometry'].to_crs(epsg = 4326)
zwalm_gpd_emma.plot()

In [ ]:
#raster_lambert = testdata.rio.reproject("epsg:31370")
#FAALT!

In [ ]:
len(zwalm_gpd_emma)

In [ ]:
zwalm_shape_epsg4326 = zwalm_gpd_emma.iloc[[15]]
type(zwalm_shape_epsg4326)
#zwalm_shape_epsg4326 = gpd.GeoDataFrame(zwalm_shape_epsg4326)
zwalm_shape_epsg4326
zwalm_shape_epsg4326.crs
zwalm_shape_epsg4326.to_file(Path(r"../data/Zwalm_shape/zwalm_shapefile_emma.shp"))


In [ ]:
zwalm_gpd_emma.iloc[[15]]

Rasterising this polygon

In [ ]:
testdata['g0vv'].shape

In [ ]:
geom = [shapes for shapes in zwalm_shape_epsg4326['geometry']]
geom

In [ ]:
testdata_rasterio = rasterio.open('netcdf:../data/g0_020m/20141118_S1A_D_110.nc:g0vh')
from rasterio.crs import CRS
crs = CRS.from_epsg(4326)
crs

In [ ]:
testdata_rasterio.transform

In [ ]:
testdata_rasterio.bounds

In [ ]:
left = testdata_rasterio.bounds.left
right = testdata_rasterio.bounds.right
top = testdata_rasterio.bounds.top
bottom = testdata_rasterio.bounds.bottom
print(left, right)
from rasterio.coords import BoundingBox
#testdata_rasterio.bounds = BoundingBox(right, bottom, left, top)
#testdata_rasterio.bounds.left = right
#testdata.bounds._right = left

In [ ]:
from rasterio.features import rasterize
rasterized_zwalm_shape = rasterize(
    zwalm_shape_epsg4326['geometry'],
    out_shape = testdata['g0vv'].shape,
    fill = 0,
    transform= testdata_rasterio.transform,
    all_touched= False,
    default_value = 1
)

In [ ]:
#rasterized_zwalm_shape_bis = rasterize(geom, out_shape = )

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(rasterized_zwalm_shape)
plt.colorbar()

In [ ]:
testdata.attrs

In [ ]:
#zwalm_gpd_emma.iloc[15,1]
zwalm_gpd_emma.loc[len(zwalm_gpd_emma)-1,"geometry"]

In [ ]:
out_image, out_transform = rasterio.mask.mask(testdata_rasterio,zwalm_shape_epsg4326['geometry'], invert = False)

In [ ]:
zwalm_shape_epsg4326['geometry']

In [ ]:
testdata_rasterio

## Zet raster om naar Tif voor masking

In [ ]:
#totiff = testdata['g0vv']
totiff = testdata
totiff

In [ ]:
totiff = totiff.rio.set_spatial_dims(x_dim = 'lon', y_dim = 'lat')
totiff = totiff.transpose('lat','lon')
totiff.rio.crs

In [ ]:
#totiff.rio.to_raster(r"../data/g0_020m_tiff/test_raster_test.tiff")

In [ ]:
#totiff.to_netcdf(r"../data/g0_020m_tiff/test_raster_test.nc", engine = 'h5netcdf')
#from osgeo import gdal
#gdal.Translate(r"../data/g0_020m_tiff/test_raster_test.tiff",
#    r"../data/g0_020m_tiff/test_raster_test.nc",
#    format = 'GeoTIFF'
#)

In [ ]:
#from osgeo import gdal
#gdal.Translate(r"../data/g0_020m_tiff/test_raster_test.tiff",
#    r'..\data\g0_020m\20141118_S1A_D_110.nc',
#    format = 'GeoTIFF'
#)

In [ ]:
testdata_rasterio_bis = rasterio.open(r"../data/g0_020m_tiff/test_raster_test.tiff")

In [ ]:
type(testdata_rasterio_bis)

In [ ]:
testdata_rasterio_bis.crs

In [ ]:
testdata_rasterio_bis.bounds

In [ ]:
out_image, out_transform = rasterio.mask.mask(testdata_rasterio_bis,zwalm_shape_epsg4326['geometry'], invert = False, crop = True)
print(type(out_image))
print(out_image.shape)
out_image_vv = out_image[0,:,:]
out_image_masked = np.ma.masked_where(out_image_vv == -9999, out_image_vv)
print(out_image_masked.shape)
plt.imshow(out_image_masked)
plt.colorbar()
plt.title('Backscatter vv [dB]')

In [ ]:
out_transform

In [ ]:
out_image.dtype

In [ ]:
len(out_image.shape)

In [ ]:
tiff_zwalm_out = rasterio.open(
    '../data/g0_020m_Zwalm/zwalm_sentinel_test_test.tiff',
    'w',
    driver = 'GTiff',
    height = out_image_vv.shape[0],
    width = out_image_vv.shape[1],
    count = out_image.shape[0],
    dtype = out_image.dtype,
    crs = testdata_rasterio_bis.crs,
    transform = out_transform,
    nodata = -9999
)
print(type(tiff_zwalm_out))
tiff_zwalm_out.write(out_image)
tiff_zwalm_out.close()

In [ ]:
type(tiff_zwalm_out)

In [ ]:
#save clipped ans masked image as geotiff, then to cdf
out_image.shape[1]


In [ ]:
out_image

In [ ]:
out_image.shape

Transform to netcdf again in GDAL

In [ ]:
inputfile =  '../data/g0_020m_Zwalm/zwalm_sentinel_test_test.tiff'
outputfile = '../data/g0_020m_Zwalm/zwalm_sentinel_test_test.nc'
from osgeo import gdal
ds = gdal.Translate(outputfile, inputfile, format = 'NetCDF')
ds

In [ ]:
test_clipped_xarray = xr.open_dataset('../data/g0_020m_Zwalm/zwalm_sentinel_test_test.nc')

In [ ]:
test_clipped_xarray['Band1'].hvplot(geo = True, cmap = 'cividis', tiles = 'OSM', frame_width = 400, alpha = 0.9, title = 'Backscatter [dB]')

In [ ]:
test_clipped_xarray.rio.write_crs('EPSG:4326', inplace = True)
